<a href="https://colab.research.google.com/github/jee365/ESAA/blob/main/%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1)
X, y = mnist['data'], mnist['target']

In [4]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)


1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [35]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [36]:
rfc_clf = RandomForestClassifier(n_estimators=100, random_state=42)
etc_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [37]:
#한번에 훈련시키기
estimators = [rfc_clf, etc_clf, svm_clf, mlp_clf]

for model in estimators:
  print('훈련:',model)
  model.fit(X_train, y_train)

훈련: RandomForestClassifier(random_state=42)
훈련: ExtraTreesClassifier(random_state=42)
훈련: LinearSVC(max_iter=100, random_state=42)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


훈련: MLPClassifier(random_state=42)


1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [38]:
from sklearn.ensemble import VotingClassifier

In [39]:
name_estimators = [('rfc',rfc_clf),('etc',etc_clf),('svm',svm_clf),('mlp',mlp_clf)]

In [40]:
voting_clf = VotingClassifier(name_estimators)

In [41]:
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier(estimators=[('rfc', RandomForestClassifier(random_state=42)),
                             ('etc', ExtraTreesClassifier(random_state=42)),
                             ('svm', LinearSVC(max_iter=100, random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [42]:
voting_clf.score(X_val,y_val)

0.9704

In [43]:
from sklearn.metrics import accuracy_score

for clf in (rfc_clf, etc_clf, svm_clf, mlp_clf):
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.9645
ExtraTreesClassifier 0.9691
LinearSVC 0.8608
MLPClassifier 0.9604


1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [44]:
del voting_clf.estimators_[2]

In [45]:
voting_clf.score(X_val, y_val)

0.9736

1-d. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [46]:
#svm 포함x
voting_clf.voting = "soft"

In [47]:
voting_clf.score(X_val, y_val)

0.97

In [49]:
#svm 포함
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier(estimators=[('rfc', RandomForestClassifier(random_state=42)),
                             ('etc', ExtraTreesClassifier(random_state=42)),
                             ('svm', LinearSVC(max_iter=100, random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [50]:
voting_clf.voting = "soft"

In [51]:
voting_clf.score(X_val, y_val) #안돼요...

AttributeError: ignored